## Fine-tuning of the classification models

!Please note, that in order to repeat a fine-tuning of the classification model, as it was done in the manuscript, you need to download the corresponding dataset, which is available in **datasets** section of https://huggingface.co/Timofey and specify all pathes to the corresponding text files, containing training and validation sequences.
<br>

**Drugs_Metabolites_Fine-Tuning_Dataset** was used for training of the Drug/Metabolites binary classifier for ANDDigest;<br>

**Genes_Proteins_Fine-Tuning_Dataset** for the Genes/Proteins classification model, etc.

In [ ]:
!nvidia-smi

In [ ]:
!pip install transformers

In [3]:
from transformers import BertTokenizer
import torch
import re

In [4]:
positive_learning_set_path = '/content/pubmed_corpus.metabolite_abstracts.BERT.learning.positive.txt' # PATH TO THE TXT FILE WITH POSITIVE EXAMPLES, THAT WILL BE USED FOR THE TRAINING OF THE MODEL
negative_learning_set_path = '/content/pubmed_corpus.metabolite_abstracts.BERT.learning.negative.txt' # PATH TO THE TXT FILE WITH NEGATIVE EXAMPLES, THAT WILL BE USED FOR THE TRAINING OF THE MODEL
positive_validation_set_path = '/content/pubmed_corpus.metabolite_abstracts.BERT.validation.positive.txt' # PATH TO THE TXT FILE WITH POSITIVE EXAMPLES, THAT WILL BE USED FOR THE ADDITIONAL VALIDATION OF THE MODEL, AFTER THE IT'S TRAINING
negative_validation_set_path = '/content/pubmed_corpus.metabolite_abstracts.BERT.validation.negative.txt' # PATH TO THE TXT FILE WITH NEGATIVE EXAMPLES, THAT WILL BE USED FOR THE ADDITIONAL VALIDATION OF THE MODEL, AFTER THE IT'S TRAINING

max_examples = 512000   # The maximal number of examples (both positive and negative) that will be used for fine-tuning
output_dir = './PubMedBERT_Drugs_Metabolites_Context_Classifier/' # PATH where the model will be saved

pos_examples = 0
counter = 0

regexp = r'^\d+\t'

dataset_texts = list()
dataset_labels = list()

# The path to your positive
input_file = open( positive_learning_set_path, 'rb' )
for line in input_file:
    line = line.decode( errors = 'replace' )
    lp = re.findall( regexp, line )[0]
    line = line.replace( lp, '' )

    if ( counter < 256000 ):
        dataset_texts.append( line )
        dataset_labels.append( 1 )
    else:
        break

    counter += 1
input_file.close()

In [5]:
counter = 0
neg_examples = 0

input_file = open( negative_learning_set_path, 'rb' )
for line in input_file:
    line = line.decode( errors = 'replace' )
    lp = re.findall( regexp, line )[0]
    line = line.replace( lp, '' )

    if ( counter < 256000 ):
        dataset_texts.append( line )
        dataset_labels.append( 0 )
    else:
        break

    counter += 1
input_file.close()

In [ ]:
tokenizer = BertTokenizer.from_pretrained( 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', do_lower_case = True )

In [ ]:
# Print the original sentence.
print( ' Original: ', dataset_texts[0] )

# Print the sentence split into tokens.
print( 'Tokenized: ', tokenizer.tokenize( dataset_texts[0] ) )

# Print the sentence mapped to token ids.
print( 'Token IDs: ', tokenizer.convert_tokens_to_ids( tokenizer.tokenize( dataset_texts[0] ) ) )

In [ ]:
max_len = 0
proper_len_count = 0
counter = 0

dataset_proper_texts = []
dataset_proper_labels = []

# For every sentence...
for sent in dataset_texts:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode( sent, add_special_tokens = True )

    # Update the maximum sentence length.
    max_len = max( max_len, len( input_ids ) )
    if ( len( input_ids ) < 512 ):
        proper_len_count += 1
        dataset_proper_texts.append( sent )
        dataset_proper_labels.append( dataset_labels[ counter ] )
    
    counter += 1

print( 'Max abstract length: ', max_len )
print( '512 and less abstracts length: ', proper_len_count )

In [ ]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in dataset_proper_texts:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation = True,
                        max_length = 512,          # Pad & truncate all sentences.
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',          # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append( encoded_dict[ 'input_ids' ] )
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append( encoded_dict[ 'attention_mask' ] )

# Convert the lists into tensors.
input_ids = torch.cat( input_ids, dim = 0 )
attention_masks = torch.cat( attention_masks, dim = 0 )
labels = torch.tensor( dataset_proper_labels )

# Print sentence 0, now as a list of IDs.
print( 'Original: ', dataset_proper_texts[0] )
print( 'Token IDs: ', input_ids[0] )

In [ ]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset( input_ids, attention_masks, labels )

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int( 0.9 * len( dataset ) )
val_size = len( dataset ) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split( dataset, [ train_size, val_size ] )

print( '{:>5,} training samples'.format( train_size ) )
print( '{:>5,} validation samples'.format( val_size ) )

In [11]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 16

train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            shuffle = True,
            batch_size = batch_size # Trains with this batch size.
        )

validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [ ]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', # Use the 12-layer PubMedBERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels, in our dataset 0 corresponds to FALSE, and 1 to TRUE.
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

model.cuda()


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list( model.named_parameters() )

print( 'The BERT model has {:} different named parameters.\n'.format( len( params ) ) )

print( '==== Embedding Layer ====\n' )

for p in params[0:5]:
    print( "{:<55} {:>12}".format( p[ 0 ], str( tuple( p[ 1 ].size() ) ) ) )

print( '\n==== First Transformer ====\n' )

for p in params[5:21]:
    print( "{:<55} {:>12}".format( p[ 0 ], str( tuple( p[ 1 ].size() ) ) ) )

print( '\n==== Output Layer ====\n' )

for p in params[-4:]:
    print( "{:<55} {:>12}".format( p[ 0 ], str( tuple( p[ 1 ].size() ) ) ) )

In [ ]:
optimizer = AdamW( model.parameters(),
                  lr = 2e-5,
                  eps = 1e-8 # args.adam_epsilon, - default value.
                )

In [15]:
from transformers import get_linear_schedule_with_warmup

epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [16]:
import numpy as np

# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [17]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
import tensorflow as tf

# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print( 'Found GPU at: {}'.format( device_name ) )
else:
    raise SystemError( 'GPU device not found' )

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device( "cuda" )

    print( 'There are %d GPU(s) available.' % torch.cuda.device_count() )

    print( 'We will use the GPU:', torch.cuda.get_device_name( 0 ) )

# If not...
else:
    print( 'No GPU available, using the CPU instead.' )
    device = torch.device( "cpu" )

In [ ]:
import random
import numpy as np

# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to( device )
        b_input_mask = batch[1].to( device )
        b_labels = batch[2].to( device )

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # The documentation for this `model` function is here: 
        # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
        # It returns different numbers of parameters depending on what arguments
        # arge given and what flags are set. For our useage here, it returns
        # the loss (because we provided labels) and the "logits"--the model
        # outputs prior to activation.
        output = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        
        loss = output[0]
        logits = output[1]

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epoch took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[ 0 ].to( device )
        b_input_mask = batch[ 1 ].to( device )
        b_labels = batch[ 2 ].to( device )
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            output = model( b_input_ids, 
                            token_type_ids = None, 
                            attention_mask = b_input_mask,
                            labels=b_labels )

            loss = output[ 0 ]
            logits = output[ 1 ]
#            print( loss )
#            print( logits )

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to( 'cpu' ).numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy( logits, label_ids )
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print( "  Accuracy: {0:.2f}".format( avg_val_accuracy ) )

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len( validation_dataloader )
    
    # Measure how long the validation run took.
    validation_time = format_time( time.time() - t0 )
    
    print( "  Validation Loss: {0:.2f}".format( avg_val_loss ) )
    print( "  Validation took: {:}".format( validation_time ) )

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print( "" )
print( "Training complete!" )

print( "Total training took {:} (h:mm:ss)".format( format_time( time.time() - total_t0 ) ) )

In [ ]:
import pandas as pd

# Display floats with two decimal places.
pd.set_option( 'precision', 2 )

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame( data = training_stats )

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index( 'epoch' )

# Display the table.
df_stats

In [ ]:
import matplotlib.pyplot as plt
#% matplotlib inline

import seaborn as sns

# Use plot styling from seaborn.
sns.set( style = 'darkgrid' )

# Increase the plot size and font size.
sns.set( font_scale = 1.5 )
plt.rcParams[ "figure.figsize" ] = ( 12, 6 )

# Plot the learning curve.
plt.plot( df_stats[ 'Training Loss' ], 'b-o', label = "Training" )
plt.plot( df_stats[ 'Valid. Loss' ], 'g-o', label = "Validation" )

# Label the plot.
plt.title( "Training & Validation Loss" )
plt.xlabel( "Epoch" )
plt.ylabel( "Loss" )
plt.legend()
plt.xticks( [ 1, 2, 3, 4 ] )

plt.show()

In [23]:
counter = 0

dataset_texts_test = []
dataset_labels_test = []

input_file = open( negative_validation_set_path, 'rb' )
for line in input_file:
  line = line.decode( errors = 'replace' )

  if ( counter < 25650 ):
    dataset_texts_test.append( line )
    dataset_labels_test.append( 0 )
  else:
    break

  counter += 1
input_file.close()

In [24]:
counter = 0

input_file = open( positive_validation_set_path, 'rb' )
for line in input_file:
  line = line.decode( errors = 'replace' )

  if ( counter < 25650 ):
    dataset_texts_test.append( line )
    dataset_labels_test.append( 1 )
  else:
    break

  counter += 1
input_file.close()

In [ ]:
max_len = 0
proper_len_count = 0
counter = 0

dataset_proper_texts_test = []
dataset_proper_labels_test = []

# For every sentence...
for sent in dataset_texts_test:

    # Tokenize the text and add `[CLS]` and `[SEP]` tokens.
    input_ids = tokenizer.encode( sent, add_special_tokens = True )

    # Update the maximum sentence length.
    max_len = max( max_len, len( input_ids ) )
    if ( len( input_ids ) < 512 ):
      proper_len_count += 1
      dataset_proper_texts_test.append( sent )
      dataset_proper_labels_test.append( dataset_labels_test[ counter ] )
    
    counter += 1

print( 'Max abstract length test set: ', max_len )
print( '512 and less abstracts length test set: ', proper_len_count )

In [26]:
import pandas as pd

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in dataset_proper_texts_test:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        truncation = True,
                        max_length = 512,           # Pad & truncate all sentences.
                        padding = 'max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append( encoded_dict[ 'input_ids' ] )
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append( encoded_dict[ 'attention_mask' ] )

# Convert the lists into tensors.
input_ids = torch.cat( input_ids, dim = 0 )
attention_masks = torch.cat( attention_masks, dim = 0 )
labels = torch.tensor( dataset_proper_labels_test )

# Set the batch size.  
batch_size = 32  

# Create the DataLoader.
prediction_data = TensorDataset( input_ids, attention_masks, labels )
prediction_sampler = SequentialSampler( prediction_data )
prediction_dataloader = DataLoader( prediction_data, sampler = prediction_sampler, batch_size = batch_size )

In [ ]:
# Prediction on test set

print( 'Predicting labels for {:,} test sentences...'.format( len( input_ids ) ) )

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions, true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple( t.to( device ) for t in batch )
  
  # Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
  
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model( b_input_ids, token_type_ids = None, 
                      attention_mask = b_input_mask )

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
  
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

print('    DONE.')

In [ ]:
from sklearn.metrics import matthews_corrcoef

matthews_set = []

# Evaluate each test batch using Matthew's correlation coefficient
print('Calculating Matthews Corr. Coef. for each batch...')

# For each input batch...
for i in range(len(true_labels)):
  
  # The predictions for this batch are a 2-column ndarray (one column for "0" 
  # and one column for "1"). Pick the label with the highest value and turn this
  # in to a list of 0s and 1s.
    pred_labels_i = np.argmax(predictions[i], axis=1).flatten()
  
  # Calculate and store the coef for this batch.  
    matthews = matthews_corrcoef( true_labels[i], pred_labels_i )                
    matthews_set.append( matthews )

In [ ]:
# Combine the results across all batches. 
flat_predictions = np.concatenate( predictions, axis = 0 )

# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax( flat_predictions, axis=1 ).flatten()

# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate( true_labels, axis = 0 )

# Calculate the MCC
mcc = matthews_corrcoef( flat_true_labels, flat_predictions )

print( 'Total MCC: %.3f' % mcc )

In [ ]:
import os

# Create output directory if needed
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

print( "Saving model to %s" % output_dir )

# Save a trained model, configuration and tokenizer using `save_pretrained()`.
# They can then be reloaded using `from_pretrained()`
model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)